In [ ]:
import numpy as np 
import pandas as pd #
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle(1).json
!kaggle competitions download -c dhi-club-tired-weekends-4thsem-1

In [ ]:
!kaggle competitions download -c dhi-club-tired-weekends-4thsem-1

In [ ]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
import numpy as np
import os
import imghdr
import cv2
import matplotlib.pyplot as plt

In [ ]:
os.listdir('/kaggle/input/dhi-dat/train_data/train_data')

In [ ]:
data = '/kaggle/input/dhi-dat/train_data/train_data'
img_exists = ['jpeg','jpg','bmp','png']

In [ ]:
for image_class in os.listdir(data):
    for image in os.listdir(os.path.join(data, image_class)):
        image_path = os.path.join(data, image_class, image)
        try:
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            #print(f"Checking: {image_path} | Detected: {tip}")  # Debugging line

            if tip not in img_exists:
                print(f'Image not in ext list {image_path}')
                # os.remove(image_path)  # Uncomment only after confirming

        except Exception as e:
            print(f'Issue with image {image_path}: {e}')
            # os.remove(image_path)


In [ ]:
import shutil

src = "/kaggle/input/dhi-dat/train_data/train_data"
dst = "/kaggle/working/train_data"

# Copy dataset to a writable location
shutil.copytree(src, dst, dirs_exist_ok=True)

print("Dataset copied successfully!")

In [ ]:
import os
from PIL import Image

def is_valid_image(image_path):
    try:
        with Image.open(image_path) as img:
            img.verify()
        return True
    except (IOError, SyntaxError):
        return False

data_dir = "/kaggle/working/train_data"

for image_class in os.listdir(data_dir):
    class_path = os.path.join(data_dir, image_class)

    for image in os.listdir(class_path):
        image_path = os.path.join(class_path, image)

        if not is_valid_image(image_path):
            print(f"Removing non-image file: {image_path}")
            os.remove(image_path)


In [ ]:
data = tf.keras.utils.image_dataset_from_directory('/kaggle/working/train_data')
data_array = data.as_numpy_iterator()
batch = data_array.next()

In [ ]:
batch[0].shape

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

scaled = batch[0]/255
scaled.max()
data = data.map(lambda x,y: (x/255, y))

scaled_array = data.as_numpy_iterator()
scaled_batch = scaled_array.next()
scaled_batch[0].max()

fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, img in enumerate(scaled_batch[0][:4]):
    ax[idx].imshow(img)
    ax[idx].title.set_text(batch[1][idx])

In [ ]:
len(data)

train_size = int(len(data)*.8)
val_size = int(len(data)*.2)
print(train_size+val_size)

train = data.take(train_size)
val = data.skip(train_size).take(val_size)

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet101

base_model = ResNet101(include_top=False, weights="imagenet", input_shape=(256, 256, 3))

for layer in base_model.layers[:int(0.05 * len(base_model.layers))]:
    layer.trainable = True

inputs = tf.keras.Input(shape=(256, 256, 3))
x = base_model(inputs, training=True)  
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.BatchNormalization()(x)  
x = tf.keras.layers.Dense(2048, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.01))(x) 
x = tf.keras.layers.Dropout(0.6)(x)  
outputs = tf.keras.layers.Dense(249, activation="softmax")(x)

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001) 

model = tf.keras.Model(inputs, outputs)
model.compile(optimizer=optimizer, loss=tf.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

#lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, verbose=1)
lr_schedule = tf.keras.optimizers.schedules.CosineDecay(
    initial_learning_rate=0.001, decay_steps=10000, alpha=0.0001
)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

model.summary()


In [ ]:
!mkdir logs1
logdir = 'logs1'
model.summary()
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
hist = model.fit(train, epochs=13, batch_size=32, validation_data=val, callbacks=[tensorboard_callback, early_stopping])


In [ ]:
fig = plt.plot()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
#fig.suptitle('loss', fontsize = 20)
plt.show()

fig = plt.plot()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
#fig.suptitle('loss', fontsize = 20)
plt.show()

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy

test_data_dir = '/kaggle/input/dhi-dat/test data'

test_data = tf.keras.utils.image_dataset_from_directory('/kaggle/input/dhi-dat/test data')
test_data_array = test_data.as_numpy_iterator()
test_batch = test_data_array.next()
print(test_batch[0].shape)

fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, img in enumerate(test_batch[0][:4]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

test_data = test_data.map(lambda x,y: (x/255, y))
scaled_test_array = test_data.as_numpy_iterator()
scaled_test_batch = scaled_test_array.next()
scaled_test_batch[0].max()

fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, img in enumerate(scaled_test_batch[0][:4]):
    ax[idx].imshow(img)
    ax[idx].title.set_text(batch[1][idx])

In [ ]:
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()

for batch in scaled_array:
  X, Y = batch
  yhat = model.predict(X)

  Y = tf.one_hot(Y, depth=249)

  pre.update_state(Y, yhat)
  re.update_state(Y, yhat)
  acc.update_state(Y, yhat)

In [ ]:
print(f'Precission:{pre.result().numpy()}, Recall:{re.result().numpy()},Accuracy:{acc.result().numpy()}')

In [ ]:
# class_names = data.class_names

# predicted_class_index = np.argmax(yhat)

# predicted_class_name = class_names[predicted_class_index]
# print(f"Predicted class index: {predicted_class_index}")
# print(f"Predicted class name: {predicted_class_name}")

# train_data = tf.keras.utils.image_dataset_from_directory('dhi_dataset/train_data/train_data')
# class_names = train_data.class_names



In [ ]:
import pandas as pd
import os
import tensorflow as tf
import numpy as np

template_df = pd.read_csv('/kaggle/input/dhi-dat/submission template.csv') 

train_data = tf.keras.utils.image_dataset_from_directory('/kaggle/input/dhi-dat/train_data/train_data')
class_names = train_data.class_names


test_data_dir = '/kaggle/input/dhi-dat/test data/test data'


for filename in os.listdir(test_data_dir):
    if filename.endswith(('.jpg', '.jpeg', '.png')):  
        image_path = os.path.join(test_data_dir, filename)

        img = tf.keras.utils.load_img(image_path, target_size=(256, 256)) 
        img_array = tf.keras.utils.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0) / 255.0 

        prediction = model.predict(img_array)
        predicted_class_index = np.argmax(prediction)
        predicted_class_name = class_names[predicted_class_index]

        template_df.loc[template_df['img_name'] == filename, 'Class Prediction'] = predicted_class_name

template_df.to_csv('updated_template.csv', index=False) 
print("Predictions saved to updated_template.csv")

In [ ]:
import shutil
shutil.move("updated_template.csv", "/kaggle/working/updated_template.csv")

In [ ]:
import os

file_path = "/kaggle/working/updated_template.csv"  

if os.path.exists(file_path):
    print("File found")
else:
    print("File NOT found")

In [ ]:
from IPython.display import FileLink

file_path = "/kaggle/working/updated_template.csv"  
FileLink(file_path)